In [ ]:
!pip install pymongo

In [ ]:
!pip install pyspark

# Load Environment Variables from .env file

We'll later pick connection strings from there. 

In [1]:
# !pip install python-dotenv
import os
from dotenv import load_dotenv
load_dotenv()

# e.g. "mongodb+srv://<username>:<password>@cluster0.mongodb.net/database.collection"
MONGO_INPUT_URI = os.getenv('MONGO_INPUT_URI', '')
MONGO_INPUT_URI != ''

True

In [2]:
from pyspark.sql import SparkSession

In [13]:
SparkSession.builder.master("local[*]").getOrCreate().stop()

Py4JError: An error occurred while calling o30.sessionState. Trace:
py4j.Py4JException: Method setConfString([class java.lang.String, class java.lang.String]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.lang.Thread.run(Thread.java:750)



In [10]:
SparkSession.builder.master("spark://spark-master:7077").getOrCreate().stop()

In [11]:
# Define Spark session
# spark.stop()
spark = (SparkSession
    .builder
    .appName("Python Spark SQL MongoDB Atlas example")
    .master('spark://spark-master:7077')
    # .master('local')
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:10.2.0')
    .config("spark.mongodb.input.uri", MONGO_INPUT_URI)
    .getOrCreate())

print(spark.version)

3.2.0


In [8]:
pipeline = "[{ $match: { '_data.t': 'initial_info' } }, { $limit: 1 }]"

df = spark.read.format('mongodb') \
    .option('database', 'request_insights') \
    .option('collection', 'requests') \
    .option('pipeline', pipeline) \
    .load()

Py4JJavaError: An error occurred while calling o96.load.
: java.lang.ClassNotFoundException: 
Failed to find data source: mongodb. Please find packages at
http://spark.apache.org/third-party-projects.html
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedToFindDataSourceError(QueryExecutionErrors.scala:443)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:670)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:720)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:210)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:174)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke

In [9]:
spark.stop()